In [1]:
import os
import zipfile
import json#libreria para trabajar conn json
import pandas as pd
from pprint import pprint 
import numpy as np
import scipy.stats  as stats #para estadistica

#para graficar bonito y facil
import plotly
from plotly.graph_objs import Scatter, Layout
import plotly.plotly as py
import plotly.graph_objs as go


salidaResultados = 'C:\\Users\\fou-f\\Desktop\\Consultoria\\python'
os.chdir(salidaResultados)# nos cambiamos de directorio
out = 'C:\\Users\\fou-f\\Desktop\\Consultoria\\python\\datosdemanda'
entrada = 'C:\\Users\\fou-f\\Desktop\\Consultoria\\python\\datosdemanda.zip'
zip_ref = zipfile.ZipFile(entrada, 'r')#descomprimimos los archivos
lista = zip_ref.extractall(out)
zip_ref.close()
archivosLista = os.listdir(out) #los archivos con los cuales trabajare
archivosLista.sort()

In [2]:
#esta parte se puede hacer mas elegante en R, sin el for yo creo que en matlab tambien es posible
all_data_sin = pd.DataFrame() #inicializamos un dataframe vacio
for archivo in range(0, len(archivosLista) ):#iteramos RECUERDENME MOSTRARLES EL CODIGO R  de esta parte en particular
    pathArchivo = out + '\\' + archivosLista[archivo]
    archivoNojason  = pd.read_json(pathArchivo)
    all_data_sin = all_data_sin.append(pd.DataFrame(data = archivoNojason), ignore_index=True)# lo junto para tener un solo dataframe
del all_data_sin['valorEnlace'] # en el correo no piden nada con 'valorEnlace' ademas son puros nulos así que la borro

In [3]:
all_data_sin.shape #verifique en R que se tengan todos los registros
all_data_sin = all_data_sin.apply(lambda x: pd.to_numeric(x, errors='coerce')) #hay cosas que lee como strings
all_data_sin.groupby('hora').count() #vemos que cada archivo es uniforme en su contenido 

,valorDemanda,valorGeneracion,valorPronostico
hora,,,
1,31,31,31
2,31,31,31
3,31,31,31
4,31,31,31
5,31,31,31
6,31,31,31
7,31,31,31
8,31,31,31
9,31,31,31


In [4]:
#obtengamos los nombres
dias = [ float((i.split('.', 3)[0])) for i in archivosLista]
dias = np.array(dias)
dias = np.repeat(dias, 24, axis=0)
meses = [ float((i.split('.', 3)[1])) for i in archivosLista]
meses = np.array(meses)
meses = np.repeat(meses, 24, axis=0)
anio = np.repeat(2016, 24*31, axis=0)
diaSemana = np.repeat('dia de la semana', 24*31)
#y los añadimos com columnas
all_data_sin = all_data_sin.assign(Dia=dias, Mes =meses, Anio = anio, DiaSemana= diaSemana)
all_data_sin.describe() #lo chido es que todos los archivos solo tienen 30 dias como maximo 

,hora,valorDemanda,valorGeneracion,valorPronostico,Anio,Dia,Mes
count,744.000000,713.000000,713.000000,744.000000,744.0,744.000000,744.000000
mean,12.500000,33162.352034,33233.032258,33792.598118,2016.0,15.354839,4.354839
std,6.926843,3147.679972,3161.135118,2754.032504,0.0,8.557487,0.478786
min,1.000000,25833.000000,25891.000000,25759.000000,2016.0,1.000000,4.000000
25%,6.750000,30339.000000,30409.000000,32227.500000,2016.0,8.000000,4.000000
50%,12.500000,33525.000000,33585.000000,35405.000000,2016.0,15.000000,4.000000
75%,18.250000,36082.000000,36133.000000,35405.000000,2016.0,23.000000,5.000000
max,24.000000,38637.000000,41340.000000,37667.000000,2016.0,30.000000,5.000000


In [5]:
#vamos por el punto 2 piden la variacion de la demanda en la semana, para eso parsee las fechas 
semana_mayo =  ['Su', 'Mo', 'Tu', 'We', 'Th', 'Fr', 'Sa']
semana_abril = ['Fr', 'Sa', 'Su', 'Mo', 'Tu', 'We', 'Th']

In [6]:
#es poco elegente pero funciona
for i in range(all_data_sin.shape[0]):
    if  all_data_sin.loc[i, 'Mes']== 5.:
        all_data_sin.loc[i, 'DiaSemana'] = semana_mayo[ int(all_data_sin.loc[i, 'Dia']) % 7 ]
    else:
        all_data_sin.loc[i, 'DiaSemana'] = semana_abril[ int(all_data_sin.loc[i, 'Dia']) % 7 ]
all_data_sin['DiaSemana'].value_counts() #lo verifique con R y si esta bien 
all_data_sin.tail(5)

,hora,valorDemanda,valorGeneracion,valorPronostico,Anio,Dia,DiaSemana,Mes
739,20,35590.0,35727.0,35235,2016,30.0,Su,4.0
740,21,36611.0,36727.0,36275,2016,30.0,Su,4.0
741,22,36892.0,36980.0,36776,2016,30.0,Su,4.0
742,23,36539.0,36619.0,36313,2016,30.0,Su,4.0
743,24,NaN,NaN,35405,2016,30.0,Su,4.0


In [7]:
#aqui hay un detalle si se suman 'valorDemanda para saber quue oferta hubo sino sale todo muy plano
resumenDemanda = all_data_sin.groupby('DiaSemana')['valorDemanda'].sum() #asi varia por semana
resumenHora = all_data_sin.groupby('hora')['valorDemanda'].sum() # asi varia por hora, la hora pico es 10 pm con un maximo de 1115646.0
resumen_dias = all_data_sin.groupby('DiaSemana')['DiaSemana'].count()#los fines de semana viernes, sabado y domingo son los dias pico 


In [8]:
resumenDemanda = pd.DataFrame(resumenDemanda)

In [9]:
resumen_dias = pd.DataFrame(resumen_dias)
resumen_dias

,DiaSemana
DiaSemana,
Fr,96
Mo,96
Sa,96
Su,96
Th,120
Tu,120
We,120


In [10]:
resumenHora = pd.DataFrame(resumenHora)
resumenHora.columns

Index(['valorDemanda'], dtype='object')

In [11]:
#la relacion entre la demanda y el pronostico, lo checamos con la correlacion de pearson que es el dafult 
demanda = all_data_sin['valorDemanda']
pronostico = all_data_sin['valorPronostico'] #es positiva 
demanda.corr(pronostico)

0.71733623458215146

In [12]:
minidata = all_data_sin[['valorDemanda', 'valorPronostico']]
minidata = minidata.dropna()
stats.pearsonr(minidata['valorDemanda'], minidata['valorPronostico']) #el p value es casi cero asi que se rechaza la hipotesis nula
#por ende la correlacion es significativa

(0.71733623458215123, 1.0893384093944085e-113)

In [13]:
#a graficar
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot({
    "data": [Scatter(x=resumenDemanda.index, y=resumenDemanda['valorDemanda'])],
    "layout": Layout(title="Demanda acumulada por día")
})


In [14]:
plotly.offline.init_notebook_mode(connected=True)
data = [go.Bar(
            x=resumen_dias.index,
            y=resumen_dias['DiaSemana']
    )]
layout = go.Layout(
    title='Concurrencia de los dias (numero de dias que aparece en los archivos)',
)
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='concurrencia')


In [15]:
plotly.offline.init_notebook_mode(connected=True)
data = [go.Bar(
            x=resumenHora.index,
            y=resumenHora['valorDemanda']
    )]
layout = go.Layout(
    title='Demanda por hora',
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='Demanda por hora')


In [16]:
#segunda parte 
salidaResultados
os.listdir(salidaResultados)
emisiones = pd.read_csv('emisiones_ejemplo.csv')
emisiones_copy = emisiones
emisiones.shape

(1048575, 8)

In [17]:
emisiones = emisiones[['Fecha', 'Latitud', 'Longitud', 'Emisiones_A', 'Emisiones_B', 'Emisiones_C']]
puntos = emisiones[[ 'Latitud', 'Longitud' ]]
puntos = puntos.drop_duplicates(['Latitud', 'Longitud'])
puntos.shape
#emisiones.loc[1]
puntos['Id'] = puntos.index
puntos

,Latitud,Longitud,Id
0,24,-99,0
1,23,-101,1
2,26,-100,2
4,25,-98,4
7,24,-100,7
10,23,-100,10
12,24,-98,12
14,25,-100,14
15,25,-99,15
16,26,-99,16


In [18]:
import plotly.plotly as py
from plotly.graph_objs import *
plotly.offline.init_notebook_mode(connected=True)
mapbox_access_token = 'pk.eyJ1IjoiZm91LWZvbyIsImEiOiJjamZ2MG10OWIwZnFrMzJxYjA1Nm9hZmNxIn0.dKCGdScNmkW38b043KmcUg'
site_lat = puntos['Latitud']
site_lon = puntos['Longitud']
data = Data([
    Scattermapbox(
        lat=site_lat,
        lon=site_lon,
        mode='markers',
        marker=Marker(
            size=17,
            color='rgb(255, 0, 0)',
            opacity=0.7
        )
    )]
)
        
layout = Layout(
    title='Área abarcada por los datos',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=23,
            lon=-101
        ),
        pitch=0,
        zoom=3,
        style='light'
    ),
)

fig = dict(data=data, layout=layout)
plotly.offline.iplot(fig, filename='Puntos')

In [19]:
resumen1 = emisiones[[ 'Latitud', 'Longitud', 'Emisiones_A', 'Emisiones_B', 'Emisiones_C']]
resumen1 = resumen1.groupby([ 'Latitud', 'Longitud']).sum()
resumen1 = resumen1.add_suffix('').reset_index()
resumen1

,Latitud,Longitud,Emisiones_A,Emisiones_B,Emisiones_C
0,23,-101,1348274,7581387,659844
1,23,-100,1343167,7551997,657331
2,23,-99,1339441,7552626,657085
3,23,-98,1340391,7510952,653961
4,24,-101,1342655,7535214,653700
5,24,-100,1329046,7472106,650239
6,24,-99,1364174,7612854,660149
7,24,-98,1339834,7516880,655766
8,25,-101,1349806,7573107,658356
9,25,-100,1348342,7549941,657398


In [20]:
plotly.offline.init_notebook_mode(connected=True)
site_lat = resumen1['Latitud']
site_lon = resumen1['Longitud']
colors = ['rgb(239,243,255)','rgb(189,215,231)','rgb(107,174,214)','rgb(33,113,181)']
parametro = .00001
data = Data([
    Scattermapbox(
        lat=site_lat,
        lon=site_lon,
        mode='markers',
        text = resumen1['Emisiones_A'],
        marker=Marker(
            size=(resumen1['Emisiones_A'])*parametro,
            color=colors[3],
            opacity=np.log(resumen1['Emisiones_A'])    )  )] )
layout = Layout(
    title='Suma de Emisiones_A',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=23,
            lon=-101 ), pitch=0, zoom=3, style='light' ))
fig = dict(data=data, layout=layout)
plotly.offline.iplot(fig, filename='Mexico')

In [21]:
plotly.offline.init_notebook_mode(connected=True)
colors = ['rgb(239,243,255)','rgb(189,215,231)','rgb(107,174,214)','rgb(33,113,181)']
alpha = .000001*2
data = Data([
    Scattermapbox(
        lat=site_lat,
        lon=site_lon,
        mode='markers',
        text = resumen1['Emisiones_B'],
        marker=Marker(
            size=(resumen1['Emisiones_B'])*alpha,
            color=colors[2],
            opacity=(resumen1['Emisiones_B'])*alpha    )  )] )
layout = Layout(
    title='Suma de Emisiones_B',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=23,
            lon=-101 ), pitch=0, zoom=3, style='light' ))
fig = dict(data=data, layout=layout)
plotly.offline.iplot(fig, filename='Mexico')


In [22]:
plotly.offline.init_notebook_mode(connected=True)
alpha = .00001*2
data = Data([
    Scattermapbox(
        lat=site_lat,
        lon=site_lon,
        mode='markers',
        text = resumen1['Emisiones_C'],
        marker=Marker(
            size=(resumen1['Emisiones_C'])*alpha,
            color='rgb(186, 33, 122)',
            opacity=(resumen1['Emisiones_C'])*alpha    )  )] )
layout = Layout(
    title='Suma de Emisiones_C',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=23,
            lon=-101 ), pitch=0, zoom=3, style='light' ))
fig = dict(data=data, layout=layout)
plotly.offline.iplot(fig, filename='Mexico')

In [23]:
emisiones2 = pd.merge(emisiones, puntos, how='outer', on=['Latitud', 'Longitud'])
import datetime as dt
a = [pd.to_datetime(ts.split()[0]) for ts in emisiones2['Fecha']]
emisiones2['Fecha'] = a
resumen2 = emisiones2.groupby([ 'Id', 'Fecha']).sum()
resumen2 = resumen2.add_suffix('').reset_index()

In [24]:
subset  = [ resumen2.loc[ resumen2['Id'] == x, resumen2.columns] for x in resumen2['Id'].unique()] 
colores = ["#FF0000FF", "#FF6000FF", "#FFBF00FF", "#DFFF00FF", "#80FF00FF", "#20FF00FF", "#00FF40FF", "#00FF9FFF",
           "#00FFFFFF", "#009FFFFF", "#0040FFFF", "#2000FFFF", "#8000FFFF", "#DF00FFFF", "#FF00BFFF", "#FF0060FF"]

In [26]:
# LA ULTIMA PARTE 
plotly.offline.init_notebook_mode(connected=True)
data =[]
ids = resumen2['Id'].unique()
for x in range(0,len(ids)-1):
    data.append(go.Scatter(
                x=pd.to_datetime(subset[x]['Fecha']),
                y=subset[x]['Emisiones_A'],
                name = "(" + str(float(puntos.loc[puntos['Id']==ids[x], 'Latitud']))+","+ str(float(puntos.loc[puntos['Id']==ids[x], 'Longitud']))+")",
                line = dict(color = colores[x]),
                opacity = 0.8))
layout = dict(
    title = "Suma de emisiones A",
    xaxis = dict(
        range = ['2015-04-01','2016-06-31'])
)

fig = dict(data=data, layout=layout)
plotly.offline.iplot(fig, filename = "Manually Set Range")